#### **Import modules**

In [9]:
import re
from typing import List

from gensim.models import Word2Vec

from utils import GeorgianWord2Vec

#### **Init word2vec model**

In [3]:
texts = [["გამარჯობა", "როგორ", "ხარ?"], ["რავი", "კარგად", "შენ?"]]

model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

model.save("word2vec.model")

## **Convert file into input format**

word2vec model gets list of list of strings as an input

In [4]:
def convert_file_into_input(file_path: str) -> List[List[str]]:
    sentences_split_regex = "\. |!|\?"
    with open(file_path, 'r') as f:
        data = f.read()
    sentences = re.split(sentences_split_regex, data)

    fn = lambda s: len(s) > 0
    return list(
        filter(fn, [list(filter(fn, [word.strip() for word in sentence.split(" ")])) for sentence in sentences]))

In [35]:
print(convert_file_into_input("test.txt"))

[['ბუნებრივი', 'ენის', 'ტექსტების', 'კომპიუტერული', 'დამუშავება', '—', 'თანამედროვე', 'საინფორმაციო', 'ტექნოლოგიების', 'ერთ-ერთი', 'მიმართულება.\nიგი', 'გულისხმობს:', 'მართლწერის', 'ავტომატურ', 'შემოწმებას;', 'მანქანურ', 'თარგმანს;', 'საძიებო', 'სისტემებს,', 'რომელთა', 'მეშვეობითაც', 'შესაძლებელია\nსიტყვის', 'სხვადასხვა', 'ფორმების', 'მოძებნა', 'ბაზაში;', 'ბუნებრივ', 'ენაზე', 'დიალოგურ', 'რეჟიმში', 'მომუშავე', 'პროგრამებს;', 'ტექსტების\nმრავალმიზნობრივ', 'მორფოლოგიურ,', 'სინტაქსურ', 'და', 'სემანტიკურ', 'ანალიზს;', 'ენის', 'სწავლებისათვის', 'განკუთვნილ', 'სავარჯიშო', 'ტრენაჟორებს', 'და\nსხვას.']]


In [6]:
print(model.wv["ავთო"])

KeyError: "Key 'ავთო' not present"

In [10]:
georgian_model = GeorgianWord2Vec()

In [11]:
georgian_model.get_model().wv["გამარჯობა"]

array([-8.7274825e-03,  2.1301615e-03, -8.7354420e-04, -9.3190884e-03,
       -9.4281426e-03, -1.4107180e-03,  4.4324086e-03,  3.7040710e-03,
       -6.4986930e-03, -6.8730675e-03, -4.9994122e-03, -2.2868442e-03,
       -7.2502876e-03, -9.6033178e-03, -2.7436293e-03, -8.3628409e-03,
       -6.0388758e-03, -5.6709289e-03, -2.3441375e-03, -1.7069972e-03,
       -8.9569986e-03, -7.3519943e-04,  8.1525063e-03,  7.6904297e-03,
       -7.2061159e-03, -3.6668312e-03,  3.1185520e-03, -9.5707225e-03,
        1.4764392e-03,  6.5244664e-03,  5.7464195e-03, -8.7630618e-03,
       -4.5171441e-03, -8.1401607e-03,  4.5956374e-05,  9.2636338e-03,
        5.9733056e-03,  5.0673080e-03,  5.0610625e-03, -3.2429171e-03,
        9.5521836e-03, -7.3564244e-03, -7.2703874e-03, -2.2653891e-03,
       -7.7856064e-04, -3.2161034e-03, -5.9258583e-04,  7.4888230e-03,
       -6.9751858e-04, -1.6249407e-03,  2.7443992e-03, -8.3591007e-03,
        7.8558037e-03,  8.5361041e-03, -9.5840869e-03,  2.4462664e-03,
      